In [1]:
import os
import requests
import time
import re
from bs4 import BeautifulSoup
import m3u8
import sys
from colorama import Fore

search_url = "https://ww.9anime2.com/search/"
watch_url = "https://ww.9anime2.com/watch/"
base = "https://ww.9anime2.com"

In [2]:
anime = input("Enter the anime : ")
search_term = "%20".join(anime.split())

Enter the anime : naruto


In [3]:
url = search_url+search_term
r = requests.get(url)
soup = BeautifulSoup(r.text, features="lxml")

In [4]:
titles = soup.find("ul", {"class": "anime-list"})
anime_list = titles.find_all("li")
title_list = ["\t".join(i.text.replace("\ndub","").replace(" Series","").replace("Special ","Special").split("\n")[3:]) for i in anime_list]

In [5]:
print(str(len(title_list)) + " Results found matching name : " + anime)
print("\Index \t Type \t Name\n")
for i in range(len(title_list)):
    print(str(i+1) + "\t " + title_list[i])

40 Results found matching name : naruto
\Index 	 Type 	 Name

1	 TV 	 Naruto 
2	 Movie 	 Naruto Shippuuden Movie 1 
3	 Movie 	 Naruto Shippuuden Movie 7: The Last 
4	 Movie 	 Boruto: Naruto the Movie (Dub) 
5	 Movie 	 Boruto: Naruto the Movie 
6	 Spring 2017 Anime 	 Boruto: Naruto Next Generations 
7	 TV 	 Boruto: Naruto Next Generations (Dub) 
8	 Movie 	 Naruto: Shippuuden Movie 6 - Road to Ninja (Dub) 
9	 Movie 	 Naruto: Shippuuden Movie 1 (Dub) 
10	 Movie 	 Naruto Shippuuden Movie 2: Kizuna 
11	 TV 	 Naruto Shippuden 
12	 TV 	 Naruto (Dub) 
13	 Special	 Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi (Dub) 
14	 Special	 Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi 
15	 Movie 	 Naruto: Shippuuden Movie 2 - Kizuna (Dub) 
16	 Movie 	 The Last: Naruto the Movie (Dub) 
17	 Movie 	 Naruto Shippuuden Movie 6: Road to Ninja 
18	 Movie 	 Naruto: Shippuuden Movie 5 - Blood Prison (Dub) 
19	 Special	 Naruto: Akaki Yotsuba no Clover wo Sagase 
20	 Movie 	 Naruto Shippuuden Mo

In [6]:
print()
choice = input("Enter anime index to download : ")

suffix = "-".join(title_list[int(choice)-1].split('\t')[1].split()).lower()
url = "https://ww.9anime2.com/watch/"+suffix
r = requests.get(url)
soup = BeautifulSoup(r.text, features="lxml")


Enter anime index to download : 1


In [7]:
episodes = soup.find("ul", {"class": "episodes"})
noe = len(episodes.find_all('li'))

In [8]:
print('\n-------------------------\n')
print("Number of downloadable episodes in the series : " + str(noe))


-------------------------

Number of downloadable episodes in the series : 220


In [9]:
choice = input("\nEnter episode(s) to be downloaded (space-separatred): ")
choice = re.sub("[^0-9]", " ", choice)


Enter episode(s) to be downloaded (space-separatred): 1


In [10]:
choices = choice.split()
for i in choices:
    if(int(i) not in range(1,noe+1)):
        print("Latest episode count is "+str(noe)+". ")
        print("Skipping episode "+i)
        choices.remove(i)
        print()

In [11]:
choices = list(sorted(set(choices)))

In [12]:
DOWNLOAD_LINKS = []
print("\nCollecting download links \n")

In [13]:
for i in choices:
    URL = url+"/"+i
    episode_page = requests.get(URL)
    soup = BeautifulSoup(episode_page.text, features="lxml")
    embed = soup.find("iframe", {"id": "playerframe"}).get('src')
    embed_url = base + embed
    embed_page = requests.get(embed_url)
    soup = BeautifulSoup(embed_page.text, features="lxml")
    data  = str(soup.find_all("script")[-1].string)
    link = re.search("(?P<url>https?://[^\s]+)", data).group("url").replace("'","")
    if link[-10:] == 'index.m3u8':
        link = link[:-10]
    DOWNLOAD_LINKS.append(link)
    print(link)
    


https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/


In [14]:
url_info = DOWNLOAD_LINKS[0]

In [15]:
file_name = 'Video 1'
url_1 = DOWNLOAD_LINKS[0]+"index.m3u8"

In [16]:
r_1 = requests.get(url_1)

In [17]:
m3u8_master = m3u8.loads(r_1.text)

In [39]:
m3u8_master.data['segments'][0]['uri']

'index0.ts'

In [42]:
m3u8_playlist_uris = [url_info+x['uri'] for x in m3u8_master.data['segments']]

In [45]:
import tqdm

In [49]:
sess = requests.Session()
with open("video.ts", 'wb') as f:
    for segment_uri in (m3u8_playlist_uris):
        print(segment_uri)
        r = sess.get(segment_uri)
        f.write(r.content)

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index0.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index1.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index2.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index3.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index4.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index5.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index6.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index7.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index8.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index9.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index10.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index11.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index12.ts
https://stream.blaze123.co/stream/6

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index110.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index111.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index112.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index113.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index114.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index115.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index116.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index117.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index118.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index119.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index120.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index121.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index122.ts
https://stre

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index218.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index219.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index220.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index221.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index222.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index223.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index224.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index225.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index226.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index227.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index228.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index229.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index230.ts
https://stre

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index326.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index327.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index328.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index329.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index330.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index331.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index332.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index333.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index334.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index335.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index336.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index337.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index338.ts
https://stre

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index434.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index435.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index436.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index437.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index438.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index439.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index440.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index441.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index442.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index443.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index444.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index445.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index446.ts
https://stre

https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index542.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index543.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index544.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index545.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index546.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index547.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index548.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index549.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index550.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index551.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index552.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index553.ts
https://stream.blaze123.co/stream/60ed3a56ca753a7f75afb0de/1080/index554.ts
https://stre

NameError: name 'a' is not defined